In [2]:
import requests
import sys
sys.path.append('../src/')
import wiki_api
import data_layer
import json
import numpy as np

In [3]:
host = "localhost"
port = "8100"

In [4]:
def get_answer(prompt):
    url = f"http://{host}:{port}?prompt={prompt}"
    response = requests.request("GET", url)
    result = json.loads(response.text)
    return result['response'] #.replace("<pad> ","").replace("</s>", "")

In [70]:
import requests
from bs4 import BeautifulSoup
import re
def get_first_paragraph(title):
    url = "https://en.wikipedia.org/wiki/" + str(title).replace(" ", "_")
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    for elem in soup.find_all(class_='mw-indicators'): # for good articles
        elem.decompose()

    summ = soup.find(class_='mw-parser-output') # Find the element or elements that contain the summary

    # cleaning of thumbnails, shortdescriptions, hatnotes and style tags
    to_clean_divs = summ.find_all('div', class_='thumb tright') + summ.find_all('div', class_=['shortdescription','hatnote']) + summ.find_all('style') + summ.find_all('link') + summ.find_all('table', class_=['metadata','sidebar']) + summ.find_all('p', 'mw-empty-elt')
    for div in to_clean_divs:
        div.decompose()

    cutoff_div = summ.find(id='toc') # Find the div element with the id "toc"
    if cutoff_div is None:
        cutoff_div = summ.find('h2')
        if cutoff_div is None:
            print(" unexpected page content on "+ url)
            return None
   
    if 'toc' in cutoff_div.parent.get('class')[0]: cutoff_div = cutoff_div.parent
    preceding_elements = cutoff_div.find_previous_siblings()
    
    return list(reversed(preceding_elements))[0].text.replace("\n","")

In [73]:
get_first_paragraph("Bessel's inequality")

"In mathematics, especially functional analysis, Bessel's inequality is a statement about the coefficients of an element x{\\displaystyle x}  in a Hilbert space with respect to an orthonormal sequence. The inequality was derived by F.W. Bessel in 1828.[1]"

## GALATICA OUTPUT

In [7]:
get_answer(f"This is a list of mathematical concepts that are required to define matrix:\n")

['This is a list of mathematical concepts that are required to define matrix:\n\n* Set\n* Function\n* Cartesian product\n* Cartesian product of sets\n* Cartesian product of functions\n* Cartesian']

In [8]:
get_answer(f"Brief definition of matrix:\n")

['Brief definition of matrix:\n\nA matrix is a rectangular array of numbers, symbols, or expressions arranged in rows and columns.\n\n# 2.2.1.1 Matrix Addition']

In [10]:
get_answer(f"A matrix is a rectangular array of numbers, symbols, or expressions arranged in rows and columns.\n\nQ:What are the mathematical objects used in the above definition?\n\nA:")

['A matrix is a rectangular array of numbers, symbols, or expressions arranged in rows and columns.\n\nQ:What are the mathematical objects used in the above definition?\n\nA:Numbers']

In [19]:
get_answer(f"Brief definition of hyperplane:\n")

['Brief definition of hyperplane:\n\nA hyperplane is a set of points in a vector space that are equidistant from a given point.\n\n\nAnswer:\n\nA hyperplane is a subspace of']

In [22]:
get_answer(f"A hyperplane is a set of points in a vector space that are equidistant from a given point.\n\nQ: What three objects are mentioned in the abstract above?\n\nA:'")

["A hyperplane is a set of points in a vector space that are equidistant from a given point.\n\nQ: What three objects are mentioned in the abstract above?\n\nA:'Vector space"]

In [25]:
get_answer(f"This is a list of concepts in the definition of Convex set:\n\n")

['This is a list of concepts in the definition of Convex set:\n\n* \\(\\forall x,y\\in S:\\forall \\lambda \\in [0,']

In [66]:
indices, wiki_concepts = data_layer.read_index_and_wiki_concepts()

In [99]:
for concept in np.random.choice(wiki_concepts, 20):
    print("CONCEPT: " + concept)
    context = get_first_paragraph(concept)
    #response = get_answer(f"Context: {context}. What concepts in this text are crucial to understand {concept}?")
    response = get_answer(f"{context}\n\nQ: What mathematical concepts are mentioned in the abstract above?\n\nA:")
    #deps = list(map(lambda x: x.strip(), response.split(",")))
    print(response)
    #print(deps)

CONCEPT: Boundary value problem
['In mathematics, in the field of differential equations, a boundary value problem is a differential equation together with a set of additional constraints, called the boundary conditions.[1] A solution to a boundary value problem is a solution to the differential equation which also satisfies the boundary conditions.\n\nQ: What mathematical concepts are mentioned in the abstract above?\n\nA: differential equations, boundary value problem']
CONCEPT: Minimal polynomial (linear algebra)
['In linear algebra, the minimal polynomial μA of an n × n matrix A over a field F is the monic polynomial P over F of least degree such that P(A) = 0.  Any other polynomial Q with Q(A) = 0 is a (polynomial) multiple of μA.\n\nQ: What mathematical concepts are mentioned in the abstract above?\n\nA: Polynomials, matrices, fields']
CONCEPT: Linear function (calculus)
['In calculus and related areas of mathematics, a linear function from the real numbers to the real numbers is

## T0PP

In [64]:
def get_answer(prompt):
    url = f"http://{host}:{port}?prompt={prompt}"
    response = requests.request("GET", url)
    result = json.loads(response.text)
    return result['response'].replace("<pad> ","").replace("</s>", "")

In [52]:
get_answer(f"Simple definition of a hyperplane:")

'<pad> A hyperplane is a plane in three-dimensional space that is perpendicular to all other planes and tangent to all other lines.</s>'

In [53]:
get_answer(f"What mathematical concepts are mentioned here? A hyperplane is a plane in three-dimensional space that is perpendicular to all other planes and tangent to all other lines.")

'<pad> plane, line, space</s>'

In [57]:
get_answer(f"In mathematics, what is matrix?")

'<pad> a two-dimensional array of elements</s>'

In [74]:
get_answer(f"Give me a detailled definition of Kernel (algebra)")

'A kernel is a function that maps a real number to another real number.'

In [62]:
get_answer(f"In linear algebra, what is matrix transpose?")

'<pad> matrix transpose is a matrix operation that transposes a matrix into its identity matrix.</s>'

In [63]:
get_answer(f"Germany // Berlin\nFrance // Paris\nRussia // Moscow\nItaly - Rome\n USA //")

'<pad> Berlin</s>'

In [174]:
import pandas as pd
def get_deps(i, concept):
    if i % 5 == 0: print(i)
    definition = get_answer(f"What is the mathematical definition of {concept}?")
    response = get_answer(f"What mathematical concepts are mentioned here: {definition}")
    deps = list(map(lambda x: x.strip(), response.split(",")))
    return deps

#concepts = np.random.choice(wiki_concepts, 100)
deps = [get_deps(i, concept) for i, concept in enumerate(wiki_concepts)]
df_concepts = pd.DataFrame({"concept": wiki_concepts, "deps": deps})
#df_concepts.to_csv("../dat/wiki/t0pp_output.csv")

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445
450
455
460
465
470
475
480
485
490
495
500
505
510
515
520
525
530
535
540
545
550
555
560
565
570
575
580
585
590
595
600
605
610
615
620
625
630
635
640
645
650
655
660
665
670
675
680
685
690
695
700
705
710
715
720
725
730
735
740
745
750
755
760
765
770
775
780
785
790
795
800
805
810
815
820
825
830
835
840
845
850
855
860
865
870
875
880
885
890
895
900
905
910
915
920
925
930
935
940
945
950
955
960
965
970
975
980
985
990
995
1000
1005
1010
1015
1020
1025
1030
1035
1040
1045
1050
1055
1060
1065
1070
1075
1080
1085
1090
1095
1100
1105
1110
1115
1120
1125
1130
1135
1140
1145
1150
1155
1160
1165
1170
1175
1180
1185
1190
1195
1200
1205
1210
1215
12

In [175]:
import wikipedia
df_concepts['search_results'] = [[wikipedia.search(f"{dep} (mathematics)") for dep in deps] for deps in df_concepts.deps.values]
df_concepts

,concept,deps,search_results
0,Vector projection,"[vector, plane, projection]","[[Vector (mathematics and physics), Vectorizat..."
1,Affine symmetric group,"[group, element, symmetry]","[[Group (mathematics), Mathematics, Gauge grou..."
2,Ideal point,"[polynomial, solution, point]","[[Polynomial, Homogeneous polynomial, Chebyshe..."
3,Probability theory,"[probability theory, statistical inference]","[[Probability theory, Probability, Probability..."
4,Trajectory,"[path, body, follow]","[[Path (topology), Hamiltonian path, Mathemati..."
...,...,...,...
1298,Standard error,[a measure of the variability of a measurement],"[[Statistical dispersion, Coefficient of varia..."
1299,Matrix decomposition,"[vector, matrix, collection]","[[Vector (mathematics and physics), Vectorizat..."
1300,Dimension,"[measure, distance, time]","[[Measure (mathematics), Mathematics, Measure,..."
1301,Orthogonality,"[line, segment, perpendicular]","[[Line (geometry), Mathematics, Phase line (ma..."


In [176]:
import re

def strip_parenthesis(title):
    return re.sub(r'\([^)]*\)', '', title).strip()

df_concepts['stripped_search'] = [[list(map(strip_parenthesis, titles)) for titles in results] for results in df_concepts.search_results]

import Levenshtein

df_concepts['dists'] = [[list(map(lambda x: Levenshtein.distance(dep, x.lower()), titles)) for dep, titles in zip(deps, results)] for deps, results in zip(df_concepts.deps, df_concepts.stripped_search)]

In [177]:
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('en')

def get_page(title):
    try:
        page = wiki_wiki.page(title)
        return page
    except wikipedia.exceptions.DisambiguationError:
        print("Could not disambiguate: " + title)
    except wikipedia.exceptions.PageError:
        print("Could not find page: " + title)

def is_disambiguation(page):
    return page.summary[0:50].find("refer") != -1

In [178]:
concept_set = set(wiki_concepts)

def accept_search_result(search_result):
    page = get_page(search_result)
    # filter disambiguation pages
    if is_disambiguation(page): return False
    outgoing_links = page.links
    return len(concept_set.intersection(outgoing_links)) > 0

def return_first_accepted(results, dists):
    for _, title in zip(dists, results): # sort by Levenshtein distances
        if accept_search_result(title): return title
    return None

In [179]:
df_concepts['dep_articles'] = [[return_first_accepted(dep_results, dep_dists) for dep_results, dep_dists in zip(deps_results, deps_dists)] for deps_results, deps_dists in zip(df_concepts.search_results, df_concepts.dists)]

  0%|          | 0/1303 [2:17:45<?, ?it/s]


In [181]:
df_concepts.to_csv("../dat/llm/df_concepts.csv")

In [192]:
# the format is needed for plotly dashboard
import json

# nodes
elements_all = {target: {"data":{"id":target,"label":target}} for target in df_concepts.concept}
for deps in df_concepts.dep_articles:
    for source in deps:
        if source not in elements_all.keys(): elements_all[source] = {"data":{"id":source,"label":source,"external": True}}
elements_all = list(elements_all.values())

# edges
for target, sources in zip(df_concepts.concept, df_concepts.dep_articles):
    for source in sources:
        elements_all.append({"data":{"source":source,"target":target}})

with open('../dat/llm/graph.json', 'w') as f:
    json.dump(elements_all, f)

In [180]:
print(list(zip(df_concepts.concept, df_concepts.deps, df_concepts.dep_articles)))

[('Vector projection', ['vector', 'plane', 'projection'], ['Vector (mathematics and physics)', 'Plane (geometry)', 'Projection (mathematics)']), ('Affine symmetric group', ['group', 'element', 'symmetry'], ['Group (mathematics)', 'Element (mathematics)', 'Symmetry in mathematics']), ('Ideal point', ['polynomial', 'solution', 'point'], ['Polynomial', 'Equation solving', 'Fixed point (mathematics)']), ('Probability theory', ['probability theory', 'statistical inference'], ['Probability theory', 'Statistical inference']), ('Trajectory', ['path', 'body', 'follow'], ['Path (topology)', 'Mathematics', 'Mathematics']), ('Radiocarbon calibration', ['process', 'sample', 'age'], ['Contact process (mathematics)', 'Sampling (statistics)', 'Mathematics']), ('Regular matrix', ['matrix', 'vector', 'row'], ['Matrix (mathematics)', 'Vector (mathematics and physics)', 'Matrix (mathematics)']), ('Linear span', ['measure', 'distance', 'point'], ['Measure (mathematics)', 'Distance', 'Fixed point (mathemati